# Example: paired-end sequencing

This is an example generated from this source
file: [`pe-example.jl`](https://github.com/bioinfologics/Pseudoseq.jl/blob/master/examples/pe-example.jl)
You are seeing the
jupyter notebook version. The corresponding online documentation page can
be found here: [`pe-example.html`](https://bioinfologics.github.io/Pseudoseq.jl/latest/examples/pe-example),
and the script can be found here: [`pe-example.jl`](./pe-example.jl)

For the simulation we are going to:

1. Create a pool of 5000 copies of a reference genome.
2. Fragment the DNA molecules in the pool, to an average length of 700bp.
3. Subsample the molecules in the pool to achieve approximatly 50x coverage.
4. Create a set of 250bp paired-end reads.
5. Apply errors to the paired-end reads at a rate of 0.001 (.1%).
6. Generate an output FASTQ file.

In [1]:
using Pseudoseq

## Using the `sequence` method

First, let's see how we do this with the `sequence` method.
The first two parameters we give to the function will be the input genome we
want to sequence, and the destination FASTQ file for output reads.
Here we are setting:
- The number of genome copies in the molecule pool to 5000.
- The average fragment size to 700bp.
- The sampling coverage to 50x.
- The read length to 250bp.
- The per base read error rate to 0.001.
- The fact we want paired-ends of fragments to be read (`paired`) to true.

In [2]:
sequence("ecoli-ref.fasta", "pe-reads.fastq"; ng = 5000, flen = 700, cov = 50, paired = true, rdlen = 250, err = 0.001)

┌ Info: - ✔ Created pool of 5000 copies of a 4639675bp genome
└ @ Pseudoseq /home/travis/build/bioinfologics/Pseudoseq.jl/src/sequence.jl:45
┌ Info: - ✔ Created pool of fragments with an average length of 700bp
└ @ Pseudoseq /home/travis/build/bioinfologics/Pseudoseq.jl/src/sequence.jl:55
┌ Info: - ✔ Subsampled pool at 50X coverage (463967 molecules)
└ @ Pseudoseq /home/travis/build/bioinfologics/Pseudoseq.jl/src/sequence.jl:62
┌ Info: - ✔ Created set of 250bp paired-end reads
└ @ Pseudoseq /home/travis/build/bioinfologics/Pseudoseq.jl/src/sequence.jl:66
┌ Info: - ✔ Applied sequencing errors at a per-base rate of 0.001
└ @ Pseudoseq /home/travis/build/bioinfologics/Pseudoseq.jl/src/sequence.jl:73
┌ Info: - ✔ Wrote 649692 paired end reads to FASTQ file
└ @ Pseudoseq /home/travis/build/bioinfologics/Pseudoseq.jl/src/reads.jl:214


## Using the Pseudoseq API

Here's how to achieve the same thing, using the Pseudoseq API. It is nessecery to
use the API if you want to do something that is not anticipated by the available
functionality of the `sequence` method: the cost of conveinience is fewer options.

Starting with a FASTA formatted file containing the genome we want to sequence,
we create a pool with 5000 copies of the genome.

In [3]:
pool = makepool("ecoli-ref.fasta", 5000)

Pool of 5000 molecules:
 All molecules are of the same size: 4639675


Next we use the fragment function to make a pool of shorter DNA molecules.

In [4]:
cutpool = fragment(pool, 700)

Pool of 33139991 molecules:
 Maximum molecule size: 12042
 Average molecule size: 700
 Minimum molecule size: 1


We need to determine the number of molecules to sample, and subsample the pool:

In [5]:
genome_size = 4639675
expected_coverage = 50
read_length = 250

N = needed_sample_size(expected_coverage, genome_size, read_length)
N = div(N, 2) # Divide by 2 as we're doing paired end sequencing.

sampledpool = subsample(cutpool, N)

Pool of 463967 molecules:
 Maximum molecule size: 9235
 Average molecule size: 698
 Minimum molecule size: 1


We now want to create a set of paired-end reads. We want our reads to be 250bp
in length.

In [6]:
pe_reads = make_reads(PairedEnd, sampledpool, 250)

Now we have some reads, we should mark positions in the reads that are destined
to be errors in the output FASTQ.

In [7]:
pe_w_errs = mark_errors(pe_reads, 0.001)

Now we have some paired end reads and have marked some positions as errors, we
can generate FASTQ files.

In [8]:
generate("pe-reads.fastq", pe_w_errs)#-

┌ Info: - ✔ Wrote 649126 paired end reads to FASTQ file
└ @ Pseudoseq /home/travis/build/bioinfologics/Pseudoseq.jl/src/reads.jl:214


*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*